In [1]:
from datetime import datetime, timedelta
from ecallisto_ng.data_download.downloader import (
    get_ecallisto_data,
    get_instrument_with_available_data,
    get_ecallisto_data_generator,
)
from ecallisto_ng.data_processing.utils import (
    elimwrongchannels,
    subtract_constant_background,
)
from ecallisto_ng.plotting.plotting import plot_spectogram
from datetime import datetime, timedelta

### Get data directly from soleil
Thanks to smart multiprocessing and fast filtering, we can get data directly from the soleil server.
We offer different functions for you to get the data you want.

In [2]:
start_datetime = datetime(2021, 5, 7, 0, 00, 0)
end_datetime = datetime(2021, 5, 7, 23, 59, 0)

In [3]:
get_instrument_with_available_data(start_datetime, end_datetime)

['ALASKA-ANCHORAGE_59',
 'ALASKA-COHOE_00',
 'ALASKA-COHOE_01',
 'ALGERIA-CRAAG_59',
 'ALMATY_58',
 'ALMATY_59',
 'AUSTRIA-Krumbach_10',
 'AUSTRIA-MICHELBACH_60',
 'AUSTRIA-OE3FLB_55',
 'AUSTRIA-OE3FLB_57',
 'AUSTRIA-UNIGRAZ_01',
 'Australia-ASSA_01',
 'Australia-ASSA_02',
 'Australia-ASSA_60',
 'Australia-LMRO_59',
 'BIR_01',
 'DENMARK_62',
 'DENMARK_63',
 'GLASGOW_59',
 'HUMAIN_59',
 'HURBANOVO_59',
 'INDIA-GAURI_59',
 'INDIA-IISERP_01',
 'INDIA-Nashik_59',
 'INDIA-OOTY_58',
 'INDIA-OOTY_59',
 'INDIA-UDAIPUR_01',
 'INDONESIA_59',
 'INPE_58',
 'INPE_59',
 'JAPAN-IBARAKI_59',
 'KASI_59',
 'KRIM_00',
 'KRIM_02',
 'MEXART_59',
 'MRO_59',
 'MRO_60',
 'MRO_61',
 'MRO_62',
 'MRT1_02',
 'MRT2_01',
 'MRT3_02',
 'MRT3_03',
 'NEWZEALAND-AUT_59',
 'NORWAY-NY-AALESUND_01',
 'NORWAY-RANDABERG_01',
 'ROSWELL-NM_57',
 'ROSWELL-NM_58',
 'ROSWELL-NM_59',
 'SOUTHAFRICA-SANSA_01',
 'SPAIN-ALCALA_04',
 'SPAIN-PERALEJOS_01',
 'SPAIN-PERALEJOS_03',
 'SPAIN-SIGUENZA_02',
 'SWISS-HB9SCT_01',
 'SWISS-HB9SCT_0

In [4]:
dfs = get_ecallisto_data(start_datetime, end_datetime, "ASSA")

In [5]:
df = dfs["Australia-ASSA_01"]

In [6]:
# Before downsamping
df.shape

(172490, 193)

In [7]:
# After downsampling, group time by 1 minute and take the max value
# See here: https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.resample.html
# The point is here that you cannot plot a lot of rows, so you need to downsample
df.resample("1min").max().shape

(1439, 193)

In [ ]:
plot_spectogram(df.resample("1min").max())

In [ ]:
# Keep only the channels 40-70. Slice the dataframe
# See: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#slicing-ranges
plot_spectogram(df.loc[:, 40:70].resample("1min").max())

In [10]:
# Generator, so that each match is downloaded and processed separately
# To allow you to process the data in chunks, save it, etc.
generator = get_ecallisto_data_generator(start_datetime, end_datetime, ["ASSA"])

In [11]:
for key, value in generator:
    print(key)

Australia-ASSA_01
Australia-ASSA_02
Australia-ASSA_60


In [12]:
# Groupby to reduce the number of rows and overwrite it
df = df.resample("1min").max()

In [ ]:
# Some spectogram editing
plot_spectogram(elimwrongchannels(subtract_constant_background(df)))

In [21]:
# Fetch interessting data, from the plot above
df = get_ecallisto_data(
    datetime(2021, 5, 7, 3, 39, 0), datetime(2021, 5, 7, 3, 42, 0), "Australia-ASSA_01"
)

In [ ]:
plot_spectogram(df)

In [ ]:
# Some spectogram editing
plot_spectogram(elimwrongchannels(subtract_constant_background(df)))

In [ ]:
# Make it prettier. .pipe in pandas is a way to chain functions
(df.pipe(elimwrongchannels).pipe(subtract_constant_background).pipe(plot_spectogram))

### Get live data

In [25]:
now = datetime.now()

start_datetime = now - timedelta(days=1)
end_datetime = now

In [26]:
df = (
    get_ecallisto_data(
        start_datetime=start_datetime,
        end_datetime=end_datetime,
        instrument_name="AUSTRIA-UNIGRAZ_01",
    )
    .resample("1min")
    .max()
)

In [ ]:
plot_spectogram(df)